*Import de módulos para el proyecto*

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

Conexión con Google Drive para acceder al archivo

In [ ]:
credito_df = pd.read_csv('/content/drive/MyDrive/Bootcamp/Proyecto Produbanco - Coding para procesamiento/creditos_historicos.csv', low_memory=False)

In [2]:
credito_df = pd.read_csv('creditos_historicos.csv', low_memory=False)

                               CREDITOS OTORGADOS - PRODUBANCO (Grupos 1-4)

En el ámbito financiero, la concesión de créditos es una de las actividades más importantes y a
la vez, una de las más riesgosas para las instituciones. La capacidad de predecir y gestionar
adecuadamente el riesgo crediticio es crucial para mantener la estabilidad financiera y
minimizar las pérdidas. Este proyecto tiene como objetivo analizar el estatus de los créditos
otorgados y determinar los factores que influyen en el incumplimiento de pago.

El análisis de datos históricos sobre la forma en que los créditos fueron otorgados y su
posterior desempeño nos permitirá identificar patrones y tendencias en el comportamiento de
los prestatarios. Con un enfoque centrado en la identificación de créditos que cayeron en
vencimiento, este proyecto busca proporcionar una comprensión profunda de los factores que
contribuyen a la morosidad.

Tenemos datos completos de todos los préstamos emitidos entre 2007 y 2015, incluido el
estado actual del préstamo (vigente, retrasado, totalmente pagado, etc.) y la última información
de pago. Las características (también conocidas como variables) incluyen la puntuación
crediticia, el número de consultas financieras, la dirección, incluidos los códigos postales y el
estado, y los cobros, entre otros. El apartado de cobros indica si el cliente ha incumplido uno o
más pagos y el equipo está intentando recuperar su dinero.

Produbanco, requiere del departamento de Business Intelligence analizar un grupo de creditos
históricos. Asuma que el mes de análisis en marzo 2019 (fecha actual).
Usted deberá cumplir con al menos los siguientes puntos:

**Definición de mal pagador**

1. Crear una definición de ‘mal_pagador’. El dataset cuenta con varias columnas que
hacen referencia los clientes que han tenido problemas de pagos. Analice y escoja una
para crear la definición de mal pagador.

Consultar tamaño del dataframe para saber numero de observaciones y series --- 2260668 observaciones, 23 variables

In [3]:
credito_df.shape

(2260668, 23)

Consultar los estados del crédito de cada cliente para crear categorías de riesgo. Ej. Fully Paid representa riesgo bajo, Late es de riesgo alto

In [4]:
credito_df['loan_status'].value_counts()

Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: loan_status, dtype: int64


Crear función que clasifique estos estados en riesgo alto o bajo

In [5]:
def status_riesgo(status):
    if status in ['Fully Paid', 'Current', 'In Grace Period']:
       return 'low risk'
    else:
       return 'high risk'

Crear nueva columna de riesgo según el estado del credito

In [6]:
credito_df['risk_status'] = credito_df['loan_status'].apply(lambda x:status_riesgo(x))

En esta nueva serie, las observaciones son 1970599 riesgo bajo y 290069 riesgo alto

In [7]:
credito_df['risk_status'].value_counts()

low risk     1970599
high risk     290069
Name: risk_status, dtype: int64

Definir función para evaluar malos pagadores según tres parámetros: riesgo_status, meses desde la última vez que cayeron en mora y las veces que estuvieron en mora en los últimos dos años.

Ej. Si el pagador tiene riesgo bajo y ha pasado más de un mes, mal pagador. Si en los últimos dos años, estuvo en mora más de 2 veces, mal pagador

In [8]:
def marca(riesgo,meses,record2yrs):
    if (riesgo == 'high risk') or (riesgo == 'low risk' and meses <6) or (record2yrs >2):
       return 1
    else:
       return 0

Crear serie aplicando la función definida anteriormente

In [9]:
credito_df['marca_mal_pagador'] = credito_df.apply(lambda x: marca(riesgo=x['risk_status'],meses=x['mths_since_last_delinq'], record2yrs=x['delinq_2yrs']), axis =1)

Existen 384384 clientes marcados como malos pagadores

In [10]:
credito_df['marca_mal_pagador'].value_counts()

0    1876284
1     384384
Name: marca_mal_pagador, dtype: int64

# **Data Quality**

*Duplicados*

1. Analice si existen valores duplicados, y si es así aborde el problema.

Para evaluar duplicados en este df revisaremos el id_cliente, ya que las otras variables podrían estar repetidas sin que signifique registro duplicado

In [11]:
credito_df['id_cliente']. duplicated().value_counts()

False    2260668
Name: id_cliente, dtype: int64

No existen duplicados, no necesita procesamiento adicional

*Nulos*

2. Analice si existen valores nulos, y si es así aborde el problema. Puede usar cualquiera
de las alternativas que vimos en clase.

isnull().any() retorna un detalle de columnas con nulos en todo el df

In [14]:
credito_df.isnull().any()

id_cliente                 False
loan_status                False
loan_amnt                  False
installment                False
term                       False
emp_title                   True
emp_length                  True
home_ownership             False
annual_inc                  True
verification_status        False
purpose                    False
addr_state                 False
delinq_2yrs                 True
next_pymnt_d                True
earliest_cr_line            True
mths_since_last_delinq      True
total_pymnt                False
recoveries                 False
collection_recovery_fee    False
last_pymnt_d                True
settlement_status           True
application_type           False
tot_hi_cred_lim             True
risk_status                False
marca_mal_pagador          False
dtype: bool

Crear view para consultar la cantidad de nulos en las series identificadas. Emplear *isnull().value_counts()*

emp_title -- 166969

emp_length -- 146907

annual_inc -- 4

delinq_2yrs -- 29

next_pymnt_d -- 1303607 nulos

earliest_cr_line -- 29

last_pymnt_d -- 2426

mths_since_last_delinq -- 1158473

settlement_status -- 2227583

tot_hi_cred_lim -- 70247

In [15]:
view= credito_df[credito_df['last_pymnt_d'].isnull()]
view.shape

(2426, 25)

Crear nuevo df eliminando las observaciones NaN de las columnas con menos nulos: annual inc, delinq 2yrs, earliest cr line.

Emplear *dropna*

In [16]:
creditos2 = credito_df.dropna(subset=['annual_inc','delinq_2yrs','earliest_cr_line'])
creditos2.shape

(2260639, 25)

Consultar los nulos identificados y ver a qué pertenecen

- Los nulos de last payment pertenecen a los incobrables, tardíos más de 30 días, incobrables y default

- Los nulos de next payment están asociados a los créditos pagados o incobrables. Asignar fecha anterior a 2000

In [17]:
nulos = creditos2.next_pymnt_d.isnull()

In [18]:
creditos2['loan_status'][nulos].value_counts()

Fully Paid     1041952
Charged Off     261655
Name: loan_status, dtype: int64

#*Tratamiento de valores NaN*

- Los nulos en columnas **emp title, emp length y settlement** son significativos, no se pueden eliminar registros. Reemplazar con valor 'unknown'

In [19]:
creditos2['emp_title'].fillna('unknown', inplace=True)
creditos2['emp_length'].fillna('unknown', inplace=True)
creditos2['settlement_status'].fillna('unknown', inplace=True)

C:\Users\vanes\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


- Fechas en **next y last payment d** reemplazar con Dic-2000 para poder separarlas de las otras fechas en las series, cuyos valores van del 2007 a 2015

In [20]:
creditos2['last_pymnt_d'].fillna('Dec-2000', inplace=True)
creditos2['next_pymnt_d'].fillna('Dec-2000', inplace=True)

- Para tratar nulos de **months since last delinq** primero consultar valores, si no existen ceros se puede asignar este valor

In [21]:
#creditos2.mths_since_last_delinq.value_counts().to_dict()

Existen 2400 observaciones con cero entonces no se puede asignar el mismo valor, rellenar con -999

In [22]:
creditos2['mths_since_last_delinq'].fillna(-999, inplace=True)

- Para **tot hi cred lim**, rellenar NaN con el promedio del resto de valores. Crear variable para después aplicar el *fillna*

In [23]:
prom_cred_mas_alto = int(creditos2['tot_hi_cred_lim'].mean())

In [24]:
creditos2['tot_hi_cred_lim'].fillna(prom_cred_mas_alto,inplace=True)

Las dos celdas siguientes se usan para confirmar nulos en el df y para ver si los valores NaN se rellenaron correctamente en las series procesadas

In [25]:
creditos2.isnull().any()

id_cliente                 False
loan_status                False
loan_amnt                  False
installment                False
term                       False
emp_title                  False
emp_length                 False
home_ownership             False
annual_inc                 False
verification_status        False
purpose                    False
addr_state                 False
delinq_2yrs                False
next_pymnt_d               False
earliest_cr_line           False
mths_since_last_delinq     False
total_pymnt                False
recoveries                 False
collection_recovery_fee    False
last_pymnt_d               False
settlement_status          False
application_type           False
tot_hi_cred_lim            False
risk_status                False
marca_mal_pagador          False
dtype: bool

In [26]:
creditos2.settlement_status.value_counts()

unknown     2227583
ACTIVE        14811
COMPLETE      13517
BROKEN         4728
Name: settlement_status, dtype: int64

Cambiar tipos de datos para facilitar el analisis

In [27]:
creditos2['annual_inc'] = creditos2['annual_inc'].astype(int)
creditos2['delinq_2yrs'] = creditos2['delinq_2yrs'].astype(int)
creditos2['mths_since_last_delinq'] = creditos2['mths_since_last_delinq'].astype(int)

C:\Users\vanes\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\vanes\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\vanes\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [28]:
creditos2.dtypes

id_cliente                   int64
loan_status                 object
loan_amnt                    int64
installment                float64
term                        object
emp_title                   object
emp_length                  object
home_ownership              object
annual_inc                   int32
verification_status         object
purpose                     object
addr_state                  object
delinq_2yrs                  int32
next_pymnt_d                object
earliest_cr_line            object
mths_since_last_delinq       int32
total_pymnt                float64
recoveries                 float64
collection_recovery_fee    float64
last_pymnt_d                object
settlement_status           object
application_type            object
tot_hi_cred_lim            float64
risk_status                 object
marca_mal_pagador            int64
dtype: object

**Convertir columnas de fecha a dt**

Los cambios se hacen inplace, el formato es mes, año (%b-%Y)

In [29]:
creditos2['next_pymnt_d'] = pd.to_datetime(creditos2['next_pymnt_d'], format = '%b-%Y')
creditos2['last_pymnt_d'] = pd.to_datetime(creditos2['last_pymnt_d'], format = '%b-%Y')
creditos2['earliest_cr_line'] = pd.to_datetime(creditos2['earliest_cr_line'],format='%b-%Y')

C:\Users\vanes\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\vanes\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\vanes\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

# **Feature Transformation**

In [36]:
creditos2.earliest_cr_line.sort_values()

596125    1933-03-01
1613074   1934-02-01
1543630   1934-04-01
1543216   1934-04-01
501898    1941-08-01
1340819   1944-01-01
2054662   1944-01-01
767741    1945-02-01
2113603   1946-01-01
1892865   1946-08-01
537492    1946-12-01
173631    1947-12-01
2073228   1948-01-01
771798    1948-01-01
1997148   1949-06-01
2185477   1949-06-01
205337    1950-01-01
955192    1950-01-01
1631407   1950-01-01
163105    1950-01-01
1626293   1950-01-01
1611740   1950-01-01
420792    1950-01-01
1134034   1950-01-01
16216     1950-01-01
230324    1950-01-01
1622814   1950-01-01
61006     1950-01-01
2086260   1950-05-01
17765     1950-07-01
             ...    
2813      2015-10-01
9902      2015-11-01
6215      2015-11-01
19667     2015-11-01
19759     2015-11-01
30457     2015-11-01
38013     2015-11-01
32818     2015-11-01
25460     2015-11-01
12093     2015-11-01
23424     2015-11-01
19978     2015-11-01
25629     2015-11-01
31672     2015-11-01
19813     2015-11-01
18249     2015-11-01
27307     201


**Creación de variables categoricas**

*Vintage del cliente*

Después de cambiar tipo de dato earliest_cr_line a datetime, ordenar los valores, consultar el más antiguo y más reciente poder definir límites de categorías de vintage de clientes

- Separar solo el año del datetime y definir limites para los bins de segmentacion, crear categorias very long, long term, established y new

In [37]:
creditos2['vintage_cliente'] = pd.cut(creditos2.earliest_cr_line.dt.year, [1900, 1969,1985,2000,2010,2020], labels=['very long term', 'long term', 'established','mid-term ','new'])

C:\Users\vanes\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


*Social Class*

Categoría basada en el nivel de ingresos del cliente

- Se empleará para determinar un promedio de ingresos y hacer más sencilla la evaluación del ratio loan/income

In [38]:
creditos2['soc_class'] = pd.cut(creditos2.annual_inc,[-1,13000,35000,65000,130000,110000000], labels=['lower','working','middle','upper-middle','upper'])

C:\Users\vanes\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Crear columna con ingresos promedio por cada categoria

In [39]:
creditos2['avg_annual_inc'] = (creditos2.groupby(['soc_class'])['annual_inc'].transform('mean')).astype(int)

C:\Users\vanes\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Creacion de ratio loan/income para verificar nivel de deuda respecto a ingresos del cliente

In [40]:
creditos2['loan/income'] = (creditos2['loan_amnt'] / creditos2['avg_annual_inc']).round(2)

C:\Users\vanes\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# **Correlation Analysis**

1. Identifique si tiene variables correlacionadas

BOX PLOT --- TERMINO DE PRESTAMO Y CANTIDAD   

solo con matriz malos pagadores

In [41]:
new_view = creditos2[['annual_inc','installment','marca_mal_pagador','mths_since_last_delinq','delinq_2yrs','recoveries','collection_recovery_fee','loan/income','debt_coverage']]

KeyError: "['debt_coverage'] not in index"

In [42]:
corr_matrix = new_view.corr(method='spearman',numeric_only=True)

NameError: name 'new_view' is not defined

In [43]:
sns.heatmap(corr_matrix, annot=True,cmap='coolwarm', center=0)

NameError: name 'corr_matrix' is not defined

2. En caso de ser así, encuentre una explicación lógica para esa correlación. Si no
encuentra un argumento, puede mencionarlo.

# **Análisis descriptivo**

1. Usando estatísticos descriptivos univariados, analice las variables que considere
relevantes y aportan información.

 loan/income para verificar nivel de deuda respecto a ingresos del cliente

In [44]:
sns.set(style="whitegrid")

# Crear el histograma usando Seaborn
plt.figure(figsize=(8, 6))
sns.histplot(creditos2['loan/income'], bins=10, binrange=(0, 1), kde=True)

# Agregar títulos y etiquetas
plt.title('Histograma de Valores con Loan/Income')
plt.xlabel('Valores')
plt.ylabel('Frecuencia')

# Ajustar el rango del eje Y
plt.xlim(0, 1)

# Mostrar el gráfico
plt.show()

AttributeError: module 'seaborn' has no attribute 'histplot'

<Figure size 576x432 with 0 Axes>

2. Usando estatísticos descriptivos bivariados, analice las variables que considere
relevantes y aportan información.

In [ ]:
# Crear la tabla de contingencia
contingencia = pd.crosstab(creditos2['soc_class'], creditos2['marca_mal_pagador'])

# Ajustar el estilo del gráfico
sns.set(style="whitegrid")

# Crear el heatmap usando Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(contingencia, annot=True, cmap="YlGnBu")

# Agregar títulos y etiquetas
plt.title('Gráfico de Contingencia')
plt.xlabel('Marca Mal Pagador')
plt.ylabel('Clase Social')

# Mostrar el gráfico
plt.show()

In [ ]:
# Crear la tabla de contingencia
tabla_cruzada = pd.crosstab(creditos2['vintage_cliente'], creditos2['marca_mal_pagador'])

# Ajustar el estilo del gráfico
sns.set(style="whitegrid")

# Crear el heatmap usando Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(tabla_cruzada, annot=True, cmap="YlGnBu", fmt='d')

# Agregar títulos y etiquetas
plt.title('Tabla Cruzada')
plt.xlabel('Marca mal Pagador')
plt.ylabel('Vintage Client')

# Mostrar el gráfico
plt.show()

In [ ]:
pd.crosstab(creditos2['emp_title'], creditos2['marca_mal_pagador'])

In [ ]:
tabla_cruzada_estado = pd.crosstab(creditos2['addr_state'], creditos2['marca_mal_pagador'], margins=True, margins_name='Total')

In [ ]:
tabla_cruzada_estado

3. Con variables categóricas, realice análisis con tablas cruzadas.

In [ ]:
sns.set(style="whitegrid")

# Crear el countplot usando Seaborn
plt.figure(figsize=(30, 8))
sns.countplot(data=creditos2, x='addr_state', hue='marca_mal_pagador')

# Agregar títulos y etiquetas
plt.title('Conteo de Mal Pagadores por Estado')
plt.xlabel('Estado')
plt.ylabel('Conteo')

In [ ]:
tabla_contingencia = pd.crosstab(creditos2['soc_class'], creditos2['marca_mal_pagador'])

In [ ]:
print(tabla_contingencia)

In [ ]:
tabla_contingencia.plot(kind='bar', stacked=True)

In [ ]:
boxplot = sns.boxplot(x='emp_title', y='marca_mal_pagador', data=creditos2)

In [ ]:
print(boxplot)

GRAFICOS ESTADISTICOS RELACIONADOS AL COMPORTAMIENTO DE MALOS PAGADORES

In [ ]:
sns.histplot(credito_df,x='risk_status',hue='loan_status')    #estados de prestamo y riesgo bajo o alto

In [ ]:
sns.displot(credito_df,x='mths_since_last_delinq',hue='risk_status',kind='kde')    #meses desde  ultimo atraso. menor para malos pagadores

In [ ]:
sns.displot(credito_df,x='delinq_2yrs',hue='risk_status',kind='kde')    #riesgo vs. record de pago ultimos dos años

In [ ]:
Analisis de malos pagadores

In [ ]:
tabla_contingencia = pd.crosstab(creditos2['soc_class'], creditos2['marca_mal_pagador'])

In [ ]:
print(tabla_contingencia)

In [ ]:
tabla_contingencia.plot(kind='bar', stacked=True)

In [ ]:
boxplot = sns.boxplot(x='emp_title', y='marca_mal_pagador', data=creditos2)

In [ ]:
print(boxplot)

In [ ]:
malos_pagadores = creditos2[creditos2['marca_mal_pagador']==1]

In [ ]:
sns.boxplot(malos_pagadores, x='term', y = 'loan_amnt', color='olive')